In [1]:
# Question 1
# Read the following data set:
# https://archive.ics.uci.edu/ml/machine-learning-databases/adult/
# Rename the columns as per the description from this file:
# https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names


import numpy as np
import pandas as pd

adultnames = pd.read_fwf('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names')
# adultnames = pd.read_fwf('adult.names.txt')
adultnames.rename(columns={adultnames.columns.values[0]:'h1'},inplace=True)

ad=adultnames['h1'].values.tolist()

# Method 1
# cols = list(adultnames.iloc[95:109].h1.str.split(":",expand=True)[0])
# print("Column Names from Adult.Names file : \n")
# print(cols)
# cols.append("salary")

# Method 2
cols=[]

for line in ad:             
    if len(str(line))>2:
        if "|" in str(line):
            a=1
        else:
            if ":" in str(line):
                word=list(line.split(":"))[0]
                cols.append(word)
cols.append("Salary")
adultdata = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',names=cols)
# adultdata = pd.read_csv('adult.data.txt',names=cols)

def trimAllColumns(df):
    trimcols = lambda x: x.strip() if type(x) is str else x
    return df.applymap(trimcols)

adultdata = trimAllColumns(adultdata)
adultdata.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [2]:
# Create a sql db from adult dataset and name it sqladb
import sqlite3 as db
from pandas.io import sql

connection = db.connect('sqladb')
cursor = connection.cursor()

query = """drop table if exists adultdata;"""
cursor.execute(query)

adultdata.to_sql('adultData', connection)

In [3]:
# 1. Select 10 records from the adult sqladb
query = "SELECT * FROM adultData LIMIT 10;"
pd.read_sql(query, connection)

,index,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Salary
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [4]:
# 2. Show me the average hours per week of all men who are working in private sector
query = """SELECT AVG([hours-per-week]) as Average FROM adultData where workclass='Private' and sex='Male';"""
pd.read_sql(query, connection)


,Average
0,42.221226


In [6]:
# 3. Show me the frequency table for education, occupation and relationship, separately

print("Frequency Table for Education")
query = """SELECT education, COUNT(education) AS frequency FROM adultData GROUP BY education LIMIT 10;"""
print(pd.read_sql(query, connection))

print("-"*100)

print("Frequency Table for occupation")
query = """SELECT occupation, COUNT(occupation) AS frequency FROM adultData GROUP BY education;"""
print(pd.read_sql(query, connection))

print("-"*100)

print("Frequency Table for relationship")
query = """SELECT relationship, COUNT(relationship) AS frequency FROM adultData GROUP BY education;"""
print(pd.read_sql(query, connection))



Frequency Table for Education
    education  frequency
0        10th        933
1        11th       1175
2        12th        433
3     1st-4th        168
4     5th-6th        333
5     7th-8th        646
6         9th        514
7  Assoc-acdm       1067
8   Assoc-voc       1382
9   Bachelors       5355
----------------------------------------------------------------------------------------------------
Frequency Table for occupation
           occupation  frequency
0   Handlers-cleaners        933
1               Sales       1175
2     Protective-serv        433
3   Machine-op-inspct        168
4   Machine-op-inspct        333
5        Craft-repair        646
6       Other-service        514
7        Tech-support       1067
8               Sales       1382
9      Prof-specialty       5355
10                  ?        413
11    Exec-managerial      10501
12    Exec-managerial       1723
13      Other-service         51
14     Prof-specialty        576
15    Protective-serv       7291
--

In [17]:
# 4. Are there any people who are married, working in private sector and having a masters
# degree

query = """SELECT * FROM adultData where workclass='Private' and education like '%Masters%' and [marital-status] like 'Married%';"""
pd.read_sql(query, connection)

,index,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Salary
0,5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
1,87,33,Private,202051,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
2,100,76,Private,124191,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
3,188,31,Private,99928,Masters,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,50,United-States,<=50K
4,198,35,Private,138992,Masters,14,Married-civ-spouse,Prof-specialty,Other-relative,White,Male,7298,0,40,United-States,>50K
5,311,34,Private,142897,Masters,14,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,7298,0,35,Taiwan,>50K
6,361,62,Private,270092,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K
7,405,41,Private,445382,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1977,65,United-States,>50K
8,435,33,Private,208405,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,>50K
9,468,49,Private,192776,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1977,45,United-States,>50K


In [18]:
# 5. What is the average, minimum and maximum age group for people working in different
# sectors

query = """SELECT workclass,max(age),min(age),AVG(age) FROM adultData group by workclass;"""
pd.read_sql(query, connection)

,workclass,max(age),min(age),AVG(age)
0,?,90,17,40.960240
1,Federal-gov,90,17,42.590625
2,Local-gov,90,17,41.751075
3,Never-worked,30,17,20.571429
4,Private,90,17,36.797585
5,Self-emp-inc,84,17,46.017025
6,Self-emp-not-inc,90,17,44.969697
7,State-gov,81,17,39.436055
8,Without-pay,72,19,47.785714


In [20]:
# 6. Calculate age distribution by country

query = """SELECT [native-country],age,count(*) FROM adultData where [native-country]<>'?' group by [native-country],age;"""
pd.read_sql(query, connection)

,native-country,age,count(*)
0,Cambodia,18,1
1,Cambodia,25,1
2,Cambodia,27,2
3,Cambodia,28,1
4,Cambodia,32,1
5,Cambodia,34,1
6,Cambodia,35,1
7,Cambodia,36,1
8,Cambodia,37,2
9,Cambodia,40,2


In [21]:
# 7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and
# 'capital-loss'

# Formula -> Net Capital Gain = Capital Gain - Capital Loss
# Ref: https://www.ato.gov.au/General/Capital-gains-tax/Working-out-your-capital-gain-or-loss/Working-out-your-net-capital-gain-or-loss/

# Adding a new column to the table
query = """ALTER TABLE adultdata ADD COLUMN netcapitalgain float;"""
cursor.execute(query)

# Calculating Net Capital gain
query = """update adultdata set netcapitalgain=[capital-gain]-[capital-loss] ;"""
cursor.execute(query)

# Fetch and see if the new fielf is appearing with calculated value
query = "SELECT * FROM adultData order by netcapitalgain desc LIMIT 10;"
print(pd.read_sql(query, connection))


   index  age         workclass  fnlwgt     education  education-num  \
0   1246   54      Self-emp-inc  166459   Prof-school             15   
1   1368   52           Private  152234       HS-grad              9   
2   1482   53      Self-emp-inc  263925       HS-grad              9   
3   1528   52           Private  118025     Bachelors             13   
4   1616   46           Private  370119   Prof-school             15   
5   1682   43           Private  176270     Bachelors             13   
6   1765   49           Private  159816     Bachelors             13   
7   1771   50           Private  171338  Some-college             10   
8   1826   22  Self-emp-not-inc  202920       HS-grad              9   
9   2103   43      Self-emp-inc  172826  Some-college             10   

       marital-status       occupation relationship                race  \
0  Married-civ-spouse   Prof-specialty      Husband               White   
1  Married-civ-spouse  Exec-managerial      Husband  Asia

In [22]:
#Task 2
# Read the following data set:
# https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

# 1. Create an sqlalchemy engine using a sample from the data set

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///:memory:', echo=True)
connection = engine.raw_connection()
Base = declarative_base()

adultdata.to_sql('adultdata', connection, if_exists='replace')

select_query = '''SELECT * FROM adultdata LIMIT 5;''' 

#Executes select query
engine.execute(select_query).fetchall()

2019-02-04 21:02:06,934 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-02-04 21:02:06,944 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 21:02:06,944 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-02-04 21:02:06,954 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 21:02:07,271 INFO sqlalchemy.engine.base.Engine SELECT * FROM adultdata LIMIT 5;
2019-02-04 21:02:07,271 INFO sqlalchemy.engine.base.Engine ()


[(0, 39, 'State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical', 'Not-in-family', 'White', 'Male', 2174, 0, 40, 'United-States', '<=50K'),
 (1, 50, 'Self-emp-not-inc', 83311, 'Bachelors', 13, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 0, 0, 13, 'United-States', '<=50K'),
 (2, 38, 'Private', 215646, 'HS-grad', 9, 'Divorced', 'Handlers-cleaners', 'Not-in-family', 'White', 'Male', 0, 0, 40, 'United-States', '<=50K'),
 (3, 53, 'Private', 234721, '11th', 7, 'Married-civ-spouse', 'Handlers-cleaners', 'Husband', 'Black', 'Male', 0, 0, 40, 'United-States', '<=50K'),
 (4, 28, 'Private', 338409, 'Bachelors', 13, 'Married-civ-spouse', 'Prof-specialty', 'Wife', 'Black', 'Female', 0, 0, 40, 'Cuba', '<=50K')]

In [23]:
# 2. Write two basic update queries

#First update query
update_query_1 = '''UPDATE adultdata SET [native-country]='USA' where [native-country]=' United-States' ;'''
engine.execute(update_query_1)


#Second update query
update_query_2 = '''UPDATE adultdata SET sex='M' where sex=' Male';'''
engine.execute(update_query_2)

#Select query 
select_query = '''SELECT * FROM adultdata LIMIT 5;''' 
engine.execute(select_query).fetchall()

2019-02-04 21:02:10,351 INFO sqlalchemy.engine.base.Engine UPDATE adultdata SET [native-country]='USA' where [native-country]=' United-States' ;
2019-02-04 21:02:10,354 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 21:02:10,366 INFO sqlalchemy.engine.base.Engine COMMIT
2019-02-04 21:02:10,371 INFO sqlalchemy.engine.base.Engine UPDATE adultdata SET sex='M' where sex=' Male';
2019-02-04 21:02:10,374 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 21:02:10,386 INFO sqlalchemy.engine.base.Engine COMMIT
2019-02-04 21:02:10,391 INFO sqlalchemy.engine.base.Engine SELECT * FROM adultdata LIMIT 5;
2019-02-04 21:02:10,394 INFO sqlalchemy.engine.base.Engine ()


[(0, 39, 'State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical', 'Not-in-family', 'White', 'Male', 2174, 0, 40, 'United-States', '<=50K'),
 (1, 50, 'Self-emp-not-inc', 83311, 'Bachelors', 13, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 0, 0, 13, 'United-States', '<=50K'),
 (2, 38, 'Private', 215646, 'HS-grad', 9, 'Divorced', 'Handlers-cleaners', 'Not-in-family', 'White', 'Male', 0, 0, 40, 'United-States', '<=50K'),
 (3, 53, 'Private', 234721, '11th', 7, 'Married-civ-spouse', 'Handlers-cleaners', 'Husband', 'Black', 'Male', 0, 0, 40, 'United-States', '<=50K'),
 (4, 28, 'Private', 338409, 'Bachelors', 13, 'Married-civ-spouse', 'Prof-specialty', 'Wife', 'Black', 'Female', 0, 0, 40, 'Cuba', '<=50K')]

In [24]:
# 3. Write two delete queries

#First delete query
update_query_1 = '''delete from adultdata where [native-country]='?' ;'''
engine.execute(update_query_1)


#Second delete query
update_query_2 = '''delete from adultdata where occupation='?';'''
engine.execute(update_query_2)

#Select query 
select_query = '''SELECT * FROM adultdata where [native-country]='?' LIMIT 5;''' 
engine.execute(select_query).fetchall()


2019-02-04 21:02:18,549 INFO sqlalchemy.engine.base.Engine delete from adultdata where [native-country]='?' ;
2019-02-04 21:02:18,557 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 21:02:18,579 INFO sqlalchemy.engine.base.Engine COMMIT
2019-02-04 21:02:18,582 INFO sqlalchemy.engine.base.Engine delete from adultdata where occupation='?';
2019-02-04 21:02:18,584 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 21:02:18,607 INFO sqlalchemy.engine.base.Engine COMMIT
2019-02-04 21:02:18,609 INFO sqlalchemy.engine.base.Engine SELECT * FROM adultdata where [native-country]='?' LIMIT 5;
2019-02-04 21:02:18,614 INFO sqlalchemy.engine.base.Engine ()


[]

In [25]:
# 4. Write two filter queries

#First filter query
filter_query1 = '''select * from adultdata where [native-country]='Cuba';'''
engine.execute(filter_query1).fetchall()



2019-02-04 21:02:54,667 INFO sqlalchemy.engine.base.Engine select * from adultdata where [native-country]='Cuba';
2019-02-04 21:02:54,684 INFO sqlalchemy.engine.base.Engine ()


[(4, 28, 'Private', 338409, 'Bachelors', 13, 'Married-civ-spouse', 'Prof-specialty', 'Wife', 'Black', 'Female', 0, 0, 40, 'Cuba', '<=50K'),
 (81, 52, 'Private', 276515, 'Bachelors', 13, 'Married-civ-spouse', 'Other-service', 'Husband', 'White', 'Male', 0, 0, 40, 'Cuba', '<=50K'),
 (638, 47, 'Self-emp-inc', 248145, '5th-6th', 3, 'Married-civ-spouse', 'Transport-moving', 'Husband', 'White', 'Male', 0, 0, 50, 'Cuba', '<=50K'),
 (701, 31, 'Private', 243605, 'Bachelors', 13, 'Widowed', 'Sales', 'Unmarried', 'White', 'Female', 0, 1380, 40, 'Cuba', '<=50K'),
 (1237, 41, 'Self-emp-not-inc', 209344, 'HS-grad', 9, 'Married-civ-spouse', 'Sales', 'Other-relative', 'White', 'Female', 0, 0, 40, 'Cuba', '<=50K'),
 (1663, 62, 'Private', 273454, '7th-8th', 4, 'Married-civ-spouse', 'Protective-serv', 'Husband', 'White', 'Male', 0, 0, 40, 'Cuba', '<=50K'),
 (2018, 39, 'Private', 505119, 'Masters', 14, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 0, 0, 40, 'Cuba', '>50K'),
 (2229, 

In [26]:

#Second filter query
filter_query2 = '''select sex,[capital-gain] from adultdata where [capital-gain] > '9999' LIMIT 5;'''
engine.execute(filter_query2).fetchall()

2019-02-04 21:03:07,294 INFO sqlalchemy.engine.base.Engine select sex,[capital-gain] from adultdata where [capital-gain] > '9999' LIMIT 5;
2019-02-04 21:03:07,304 INFO sqlalchemy.engine.base.Engine ()


[('Female', 14084),
 ('Female', 14344),
 ('Male', 15024),
 ('Female', 14084),
 ('Female', 15024)]

In [27]:
# Write two function queries
function_query1 = '''select workclass,sum([capital-gain]),sum([capital-loss]) from adultdata where workclass<>'?' group by workclass LIMIT 5;'''
engine.execute(function_query1).fetchall()

2019-02-04 21:03:16,152 INFO sqlalchemy.engine.base.Engine select workclass,sum([capital-gain]),sum([capital-loss]) from adultdata where workclass<>'?' group by workclass LIMIT 5;
2019-02-04 21:03:16,162 INFO sqlalchemy.engine.base.Engine ()


[('Federal-gov', 784879, 105363),
 ('Local-gov', 1714019, 228038),
 ('Private', 19608520, 1774623),
 ('Self-emp-inc', 5166742, 164864),
 ('Self-emp-not-inc', 4780923, 288815)]

In [28]:
function_query2 = '''select max([hours-per-week]) ,min([hours-per-week]),avg([hours-per-week]) from adultdata;'''
engine.execute(function_query2).fetchall()

2019-02-04 21:03:19,154 INFO sqlalchemy.engine.base.Engine select max([hours-per-week]) ,min([hours-per-week]),avg([hours-per-week]) from adultdata;
2019-02-04 21:03:19,154 INFO sqlalchemy.engine.base.Engine ()


[(99, 1, 40.93123798156621)]